In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/run_ner.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/utils_ner.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/tasks.py

!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..

In [ ]:
!pip install pyarrow --upgrade

In [ ]:
import transformers

In [ ]:
# !mkdir data
# aynı dizinde data klasöründe train,test,dev tsv dosyaları yükleniyor.
# eğer text dosyaları varsa hazır, direk run_ner.py çalıştırılabilir.

blankLineIndicator = "BlankLineIndicator"
blank = ""
firstColumnIndex = 0
secondColumnIndex = 1

In [ ]:
!unzip NERdata.zip -d data

In [ ]:
train_dev_tsv = []

with open('./data/NCBI-disease/train_dev.tsv', 'r') as f:
  train_dev_pd = f.readlines()


for row in train_dev_pd:
    row = row.split('\n')[0].split('\t')
    #Token Sütun İsmi
    if row:
      pass
    if row[firstColumnIndex] == '':
        train_dev_tsv.append(blank)
    else:           #Token Sütun İsmi
       #print(row)
       train_dev_tsv.append(row[firstColumnIndex] +  " " + row[secondColumnIndex])


In [ ]:
test_tsv = []

with open('./data/NCBI-disease/test.tsv', 'r') as f:
  test_pd = f.readlines()


for row in test_pd:
    row = row.split('\n')[0].split('\t')
    #Token Sütun İsmi
    if row:
      pass
    if row[firstColumnIndex] == '':
        test_tsv.append(blank)
    else:           #Token Sütun İsmi
       #print(row)
       test_tsv.append(row[firstColumnIndex] +  " " + row[secondColumnIndex])


In [ ]:
dev_tsv = []

with open('./data/NCBI-disease/dev.tsv', 'r') as f:
  test_pd = f.readlines()


for row in test_pd:
    row = row.split('\n')[0].split('\t')
    #Token Sütun İsmi
    if row:
      pass
    if row[firstColumnIndex] == '':
        dev_tsv.append(blank)
    else:           #Token Sütun İsmi
       #print(row)
       dev_tsv.append(row[firstColumnIndex] +  " " + row[secondColumnIndex])


In [ ]:
train_dev_pd[0], train_dev_tsv[0]

In [ ]:
l = []
for item in train_dev_tsv:
  try:
    item = item.split()[1]
    if item != 'B' and item != 'I' and item != 'O':
      print(item)
    l.append(item)
  except:
    pass

l = set(l)
print(l)

with open('labels.txt', 'w') as f:
  for item in list(l):
    f.write(item + '\n')


In [ ]:
!cut -f2 BC2GM/train.tsv | sort | uniq

In [ ]:
def create_txt(file_name, lines):
    file = open(file_name, 'w') 
    for line in lines:
        file.write(line + "\n") 
    file.close()

#create_txt("./data/train.txt",train_tsv)
create_txt("data/NCBI-disease/train.txt", train_dev_tsv)
create_txt("data/NCBI-disease/test.txt", test_tsv)
create_txt("data/NCBI-disease/dev.txt", dev_tsv)
# txt file'a çeviriyoruz

In [ ]:
# !cat data/NCBI-disease/train.tsv | tr "\t" " " | head -10

In [ ]:
#labels.txt -> unique varlık ismi sınıflarının olduğu text dosyası

In [ ]:
OUTPUT_DIR = "electra-ner"

In [ ]:
!python3 run_ner.py --data_dir ./data/NCBI-disease/ \
--labels ./labels.txt \
--model_name_or_path enelpi/med-electra-small-discriminator \
--output_dir $OUTPUT_DIR \
--max_seq_length 192 \
--num_train_epochs 3 \
--per_device_train_batch_size 16 \
--overwrite_output_dir \
--save_steps 10000 \
--seed 41 \
--do_train \
--do_eval \
--do_predict

In [ ]:
!nvidia-smi